In [1]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import pickle
from sklearn import naive_bayes, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


<h1><strong> AIM: phân loại -keyword - tóm tắt(kmeans cho tiện) - khai phá thông tin liên quan(association rule)</strong></h1>

<h2>lấy dữ liệu đã được xử lý</h2>
<p>dữ liệu phục vụ cho mô hình là một csr matrix, hay nói cách khác là một tập hợp gồm n phần tử, mỗi phần tử là một vector p chiều</p>
<p>mỗi chiều của vector là một từ trong tiếng việt, bao gồm cả từ đơn và từ ghép</p>
<p>file .pkl dùng để lưu lại các object(list, tuple ...) trong python</p>
<p><strong>count_vect_data lưu lại một ma trận có 40000 cột và số hàng tương ứng với số văn bản, mỗi phần tử là số lần xuất hiện của từ trong văn bản</strong><p>
<p><strong>tfidf như cont_vect, nhưng giá trị phần tử là tần suất thay vì số lần xuất hiện</strong></p>
<p><strong>tfidf_svd là tfidf nhưng đã giảm số chiều(số cột)</strong></p>

In [2]:
X_data_count = pickle.load(open("count_vect_data.pkl", "rb"))
X_test_count = pickle.load(open("count_vect_test.pkl", "rb"))
X_data_tfidf_svd = pickle.load(open("X_data_tfidf_svd.pkl", "rb"))
X_test_tfidf_svd = pickle.load(open("X_test_tfidf_svd.pkl", "rb"))
y_data = pickle.load(open("raw_data/y_data.pkl", "rb"))
y_test = pickle.load(open("raw_data/y_test.pkl", "rb"))
X_data = pickle.load(open("raw_data/X_data.pkl", "rb"))
X_test = pickle.load(open("raw_data/X_test.pkl", "rb"))
X_data_tfidf = pickle.load(open("X_data_tfidf.pkl", "rb"))

Feature extraction: Extract relevant features from the text data, such as n-grams, parts of speech, or named entities, that can be used to identify sentiment and opinion.
this step was implemented after converting tfidf matrix

<h2>đánh nhãn cho các lớp, do model hoạt động dựa trên dữ liệu số</h2>


In [3]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ # kết quả: array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
                 #                 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
                 #                 'Vi tinh'], dtype='<U16')

#cell nay bo cx dc, cac thuat toan van chay binh thuong

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

<h2>build hàm huấn luyện chung để cho các model để thuận tiện thay đổi model</h2>

In [21]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)

        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
    #% correct classfied
    print('accuracy score mertric:')
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))
    #The best value is 1 and the worst value is 0.
    # print('precision score mertric:')
    # print("Validation precision: ", metrics.precision_score(val_predictions, y_val))
    # print("Test precision: ", metrics.precision_score(test_predictions, y_test))



<h1>Tiến hành build các model</h1>

<h2>1, Naive-bayes</h2>

In [22]:
naivBayesType = naive_bayes.MultinomialNB()
train_model(naivBayesType, X_data_count, y_data, X_test_count, y_test, is_neuralnet=False)

accuracy score mertric:
Validation accuracy:  0.9164490861618799
Test accuracy:  0.7852889667250438


<h2>2, SVM</h2>
<p>svm và knn test dữ liệu có số chiều nhỏ<p>

In [23]:
from sklearn import svm
svmModel = svm.SVC()
train_model(svmModel, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

accuracy score mertric:
Validation accuracy:  0.9295039164490861
Test accuracy:  0.7957968476357268


<h2>3, KNN</h2>


In [24]:
from sklearn import neighbors
NB_model = neighbors.KNeighborsClassifier()
train_model(NB_model, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

accuracy score mertric:
Validation accuracy:  0.8459530026109661
Test accuracy:  0.6612959719789843


<h2>decision tree</h2>
<p>min_samples_split:2 if you are splitting a node at minimum it should have 2 records which after splitting into two nodes will give 1 record each which is specified by the min_samples_leaf.:( minimum number of samples required to split an internal node.)</p>
<p>split will not happen if there are less than a certain number of records specified by min_samples_split in a node.(minimum number of samples required to be at a leaf node)</p>
<strong>these parameters help control the depth, size, and shape of the tree</strong>
<p>sample: row-theo matrix</p>

In [25]:
from sklearn import tree 
DecisionTree_model = tree.DecisionTreeClassifier(criterion='gini')#dung entropy co ve do chinh xac khong bawng gini
train_model(DecisionTree_model, X_data_count, y_data, X_test_count, y_test, is_neuralnet=False)

accuracy score mertric:
Validation accuracy:  0.6762402088772846
Test accuracy:  0.5516637478108581


<h2>random forest</h2>

In [26]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(criterion='entropy')
train_model(RF_model, X_data_count, y_data, X_test_count, y_test, is_neuralnet=False)


accuracy score mertric:
Validation accuracy:  0.8642297650130548
Test accuracy:  0.6854640980735551


<h2> sử dụng model </h2>

<p>tạo mẫu</p>

các tư thế xử lý dữ liệu

In [27]:
from sklearn.decomposition import TruncatedSVD
#chuyển data về kiểu tfidf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
#chuyển data về kiểu giảm số chiều

#nếu không có random state, mỗi lần chạy kết quả sẽ khác nhau
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)
#chuyển data về kiểu đếm số lần xuất hiện 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)



CountVectorizer(token_pattern='\\w{1,}')

<p>preprocessing theo kiểu tiếng việt</p>

In [28]:
test = ["ManCity đang có phong độ tốt trước trận tứ kết lượt đi Champions League trên sân nhà gặp Bayern. Đoàn quân của Pep Guardiola hiểu rằng họ cần có một kết quả tốt trước khi làm khách trên sân đối thủ ở lượt về, vì vậy ManCity chủ động dâng cao đội hình tấn công gây sức ép lên đối phương ngay sau tiếng còi khai cuộc. Phút 5, De Bruyne chuyền bóng cho Haaland từ bên trái. Cầu thủ người Na Uy bứt tốc nhanh và dứt điểm uy lực từ bên trái vòng cấm địa, bóng đi vọt xà. 9 phút sau, Haaland khiến cổ động viên Bayern thót tim khi lao vào rất nhanh tranh bóng với thủ thành Sommer ngay trên vạch vôi. Thủ môn đội khách xử lý chậm, anh phá bóng đập chân Haaland nhưng may mắn trái bóng đã văng ra biên. Haaland thi đấu nổi bật trên hàng công của Man City, các đồng đội cũng tập trung nhiều bóng cho anh. Phút 22, Haaland thoải mái dứt điểm ở đầu vòng cấm địa. Tiếc rằng cú sút của anh hơi nhẹ, bóng đi chìm và hướng thẳng vào vị trí của Sommer nên không gây nguy hiểm."]
lines = gensim.utils.simple_preprocess(test[0])#loại bỏ các phần không liên quan, ex: dấu chấm, dấu phẩy
lines = " ".join(lines)
lines = ViTokenizer.tokenize(lines)
test[0] = lines
print(lines)

mancity đang có phong_độ tốt trước trận tứ_kết lượt đi champions league trên sân_nhà gặp bayern đoàn quân của pep guardiola hiểu rằng họ cần có một kết_quả tốt trước khi làm_khách trên sân đối_thủ lượt về vì_vậy mancity chủ_động dâng cao đội_hình tấn_công gây sức_ép lên đối_phương ngay sau tiếng còi khai_cuộc phút de bruyne chuyền bóng cho haaland từ bên trái cầu_thủ người na uy bứt tốc nhanh và dứt_điểm uy_lực từ bên trái vòng cấm_địa bóng đi vọt xà phút sau haaland khiến cổ_động_viên bayern thót_tim khi lao vào rất nhanh tranh bóng với thủ_thành sommer ngay trên vạch vôi thủ_môn đội khách xử_lý chậm anh phá bóng đập chân haaland nhưng may_mắn trái bóng đã văng ra biên haaland thi_đấu nổi_bật trên hàng công của man city các đồng_đội cũng tập_trung nhiều bóng cho anh phút haaland thoải_mái dứt_điểm đầu vòng cấm_địa tiếc rằng cú sút của anh hơi nhẹ bóng đi chìm và hướng thẳng vào vị_trí của sommer nên không gây nguy_hiểm


<p>chuyển kiểu dữ liệu tf-idf</p>

In [29]:

example_count_type = count_vect.transform(test)

example = tfidf_vect.transform(test)

example_truncated = svd.transform(example)

In [30]:
print(len(example_truncated))

1


dự đoán

In [31]:
print("kết quả của naive-bayes model: {}".format(naivBayesType.predict(example_count_type)))
print("kết quả của SVM model: {}".format(svmModel.predict(example_truncated)))
print("kết quả của KNN model: {}".format(NB_model.predict(example_truncated)))
print("kết quả của DT model: {}".format(DecisionTree_model.predict(example_count_type)))
print("kết quả của RF model: {}".format(RF_model.predict(example_count_type)))

kết quả của naive-bayes model: ['The thao']
kết quả của SVM model: ['The thao']
kết quả của KNN model: ['The thao']
kết quả của DT model: ['The thao']
kết quả của RF model: ['The thao']


<h2>keyword extraction</h2>

In [32]:
feature_names = tfidf_vect.get_feature_names_out()
feature_scores = example.toarray()[0]
# Sort the features by their scores
sorted_indices = feature_scores.argsort()[::-1]
# Print the top 5 features and their scores
for i in range(5):
    print(feature_names[sorted_indices[i]], feature_scores[sorted_indices[i]])

bóng 0.40895724566074687
bayern 0.25206173354962336
trái 0.20103160734484218
cấm_địa 0.1936749761034522
dứt_điểm 0.1706720230636951


<h2> text mining</h2>
<strong> cần tối ưu - cần tiền xử lí văn bản </strong>
<p> hiện chưa thể làm việc với văn bản lớn </p>
<p> field: keyword extraction, sentiment analysis, and recommendation systems.</p>

Keyword extraction: We can treat each document in a corpus as a transaction and each word as an item, and use the Apriori algorithm to identify frequent word patterns that occur across multiple documents. These frequent word patterns can be used as keywords to summarize the key themes or topics in the corpus.

In [33]:
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules
# import pandas as pd
# # Define the documents
# documents = X_data

# # Tokenize the documents
# tokenized_documents = [document.lower().split() for document in documents]

# # Encode the tokenized documents
# te = TransactionEncoder()
# te_ary = te.fit(tokenized_documents).transform(tokenized_documents)
# df = pd.DataFrame(te_ary, columns=te.columns_)

# # Find frequent itemsets using Apriori algorithm
# frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# # Generate association rules
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

# # Print the association rules
# #print(rules)

<h2>text summarization</h2>

<p>example:</p>

In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
text = ["Hồ Chí Minh luôn luôn đặt những vấn đề kinh tế trong mối quan hệ chặt chẽ với những vấn đề chính trị - xã hội: Chúng ta giành được tự do, độc lập rồi mà dân cứ chết đói, chết rét, thì tự do, độc lập cũng không làm gì. Dân chỉ biết rõ giá trị của tự do, của độc lập khi mà dân được ăn no, mặc đủ. Do vậy, ngay sau khi giành chính quyền về tay nhân dân, Người đã kêu gọi nhân dân cả nước tích cực tăng gia sản xuất, quyết tâm diệt giặc dốt và chỉ rõ trách nhiệm Nếu dân đói, Đảng và Chính phủ có lỗi. Với một nước nông nghiệp, Người đưa ra cơ cấu kinh tế nông - công nghiệp; xem nông nghiệp là mặt trận hàng đầu đảm bảo an ninh lương thực để công nghiệp hoá và là hậu phương vững chắc cho sự nghiệp cách mạng. Nền kinh tế xã hội chủ nghĩa phải được tạo lập trên cơ sở chế độ sở hữu công cộng về tư liệu sản xuất. Hồ Chí Minh là người sớm đưa ra chủ trương phát triển cơ cấu kinh tế nhiều thành phần trong thời kỳ quá độ lên chủ nghĩa xã hội ở nước ta. Chủ tịch Hồ Chí Minh rất chú trọng đến công tác nghiên cứu và phổ biến khoa học kỹ thuật phục vụ sản xuất. Người coi trọng vấn đề quản lý, hạch toán kinh tế, cho đó là chìa khoá phát triển kinh tế quốc dân. Người đề xuất chính sách mở cửa và hợp tác với các nước để thu hút ngoại lực và phát huy nội lực. Người cũng đã bước đầu đề cập đến vấn đề khoán trong sản xuất. Hồ Chí Minh đã đưa ra một định nghĩa về văn hoá với nghĩa rộng: Vì lẽ sinh tồn cũng như mục đích của cuộc sống, loài người mới sáng tạo và phát triển ra những ngôn ngữ, chữ viết, đạo đức, pháp luật, khoa học, tôn giáo, văn học, nghệ thuật, những công cụ sinh hoạt hàng ngày về mặc, ăn, ở và các phương tiện sử dụng. Toàn bộ những sáng tạo và phát minh đó tức là văn hoá. Văn hoá là sự tổng hợp của mọi phương thức sinh hoạt cùng với biểu hiện của nó mà loài người đã sản sinh ra nhằm thích ứng nhu cầu đời sống và đòi hỏi của sự sinh tồn. Theo Hồ Chí Minh, văn hoá có nhiệm vụ chủ yếu là bồi dưỡng con người có tư tưởng đúng và tình cảm cao đẹp; nâng cao dân trí, nghĩa là đề cập tới chức năng giáo dục của văn hoá; bồi dư"]
lines = ViTokenizer.tokenize(text[0])
text[0] = lines

<h3>1, split sentence</h3>

In [36]:
sentences = nltk.sent_tokenize(text[0])

# In ra các câu được tách
for sentence in sentences:
    print(sentence)

Hồ_Chí_Minh luôn_luôn đặt những vấn_đề kinh_tế trong mối quan_hệ chặt_chẽ với những vấn_đề chính_trị - xã_hội : Chúng_ta giành được tự_do , độc_lập rồi mà dân cứ chết đói , chết rét , thì tự_do , độc_lập cũng không làm gì .
Dân chỉ biết rõ giá_trị của tự_do , của độc_lập khi mà dân được ăn no , mặc đủ .
Do_vậy , ngay sau khi giành chính_quyền về tay nhân_dân , Người đã kêu_gọi nhân_dân cả nước tích_cực tăng_gia_sản_xuất , quyết_tâm diệt giặc dốt và chỉ rõ trách_nhiệm Nếu dân đói , Đảng và Chính_phủ có lỗi .
Với một nước nông_nghiệp , Người đưa ra cơ_cấu kinh_tế nông - công_nghiệp ; xem nông_nghiệp là mặt_trận hàng_đầu đảm_bảo_an_ninh lương_thực để công_nghiệp_hoá và là hậu_phương vững_chắc cho sự_nghiệp cách_mạng .
Nền kinh_tế xã_hội chủ_nghĩa phải được tạo_lập trên cơ_sở chế_độ sở_hữu công_cộng về tư_liệu sản_xuất .
Hồ_Chí_Minh là người sớm đưa ra chủ_trương phát_triển cơ_cấu kinh_tế nhiều thành_phần trong thời_kỳ quá_độ lên chủ_nghĩa xã_hội ở nước ta .
Chủ_tịch Hồ_Chí_Minh rất chú_tr

<h3>2, change sentence to vector of word</h3>

In [37]:
sentences_count_type = count_vect.transform(sentences)
print(type(sentences_count_type))

<class 'scipy.sparse._csr.csr_matrix'>


<h3>3, Build Kmeans model </h3>

In [38]:
from sklearn.cluster import KMeans

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(sentences_count_type)

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [39]:
pickle.dump(kmeans, open("kmeans_object.pkl", "wb"))

<h3>4, summary</h3>

<p>điểm lấy ra để đại diện cho các câu là điểm gần centroid nhất</p>

In [40]:
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np 

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    #dùng để sắp xếp thứ tự các câu, = trung bình thứ tự xuất hiện
    avg.append(np.mean(idx))
#chọn câu đại diện là câu gần trung tâm nhất
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentences_count_type)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [41]:
print(summary)

Hồ_Chí_Minh luôn_luôn đặt những vấn_đề kinh_tế trong mối quan_hệ chặt_chẽ với những vấn_đề chính_trị - xã_hội : Chúng_ta giành được tự_do , độc_lập rồi mà dân cứ chết đói , chết rét , thì tự_do , độc_lập cũng không làm gì . Toàn_bộ những sáng_tạo và phát_minh đó tức_là văn_hoá . Hồ_Chí_Minh đã đưa ra một định_nghĩa về văn_hoá với nghĩa rộng : Vì lẽ sinh_tồn cũng như mục_đích của cuộc_sống , loài_người mới sáng_tạo và phát_triển ra_những ngôn_ngữ , chữ_viết , đạo_đức , pháp_luật , khoa_học , tôn_giáo , văn_học , nghệ_thuật , những công_cụ sinh_hoạt hàng ngày về mặc , ăn , ở và các phương_tiện sử_dụng .


<h1> LDA algorithm for text's information extraction</h1>

In [42]:
print(X_data_count)

  (0, 2304)	1
  (0, 2538)	4
  (0, 2852)	2
  (0, 2943)	2
  (0, 3933)	4
  (0, 4747)	1
  (0, 5602)	3
  (0, 5728)	1
  (0, 5811)	2
  (0, 5955)	1
  (0, 6015)	2
  (0, 6023)	1
  (0, 6060)	1
  (0, 6289)	1
  (0, 6639)	1
  (0, 8033)	2
  (0, 8228)	2
  (0, 9564)	2
  (0, 9602)	2
  (0, 9729)	1
  (0, 10144)	1
  (0, 10271)	2
  (0, 10441)	1
  (0, 10969)	1
  (0, 10989)	1
  :	:
  (3822, 16627)	3
  (3822, 16782)	1
  (3822, 18005)	1
  (3822, 19508)	1
  (3822, 19576)	2
  (3822, 20081)	1
  (3822, 21184)	1
  (3822, 21329)	1
  (3822, 22148)	3
  (3822, 26541)	1
  (3822, 26984)	1
  (3822, 27772)	1
  (3822, 28617)	1
  (3822, 29028)	1
  (3822, 30210)	2
  (3822, 31899)	1
  (3822, 31944)	1
  (3822, 32366)	2
  (3822, 32681)	2
  (3822, 32683)	1
  (3822, 32695)	1
  (3822, 34021)	1
  (3822, 34477)	1
  (3822, 34669)	1
  (3822, 35705)	1


In [43]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
# Create a corpus of text documents


# Create a CountVectorizer object to convert the text to a matrix of word counts


# Convert the corpus to a matrix of word counts


# Apply NMF to the count matrix to factorize it into non-negative components
# nmf = NMF(n_components=300)
# X_nmf = nmf.fit_transform(X_data_count)

# Create an LDA object with 10 topics
lda = LatentDirichletAllocation(n_components=10)

# Fit the LDA model to the reduced data
X_data_count_2 = X_data_count[0:200]
lda.fit(X_data_count_2)

# Print the top 5 words for each topic
feature_names = count_vect.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-6:-1]]))

Topic #0:
và các của có cho
Topic #1:
là vé người có một
Topic #2:
polytechnic tài láng helsinki thấn
Topic #3:
tàu appf sự_cố chìm cựu_binh
Topic #4:
và có đã các của
Topic #5:
phù_điêu hiếu_học tấm chiến_đấu đúc
Topic #6:
là các của và có
Topic #7:
ve_chai rượu thú thừa phát
Topic #8:
núi nhậu cấm nướng đồng_bằng
Topic #9:
ford giữ mù hắn thủ_đoạn
